In [121]:
%pip install rubik-cube numpy matplotlib pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [122]:
from rubik.cube import Cube

def colored_cube(cube: Cube):
  cube_str = ""
  for char in cube.__str__():
      if(char == "O"):
        cube_str += "\033[38;5;214mO\033[0m"
      elif(char == "Y"):
        cube_str += "\033[38;5;11mY\033[0m"
      elif(char == "W"):
        cube_str += "\033[38;5;15mW\033[0m"
      elif(char == "G"):
        cube_str += "\033[38;5;10mG\033[0m"
      elif(char == "B"):
        cube_str += "\033[38;5;12mB\033[0m"
      elif(char == "R"):
        cube_str += "\033[38;5;9mR\033[0m"
      else:
        cube_str += char
  
  return cube_str

c = Cube("OOOOOOOOOYYYWWWGGGBBBYYYWWWGGGBBBYYYWWWGGGBBBRRRRRRRRR")

print(colored_cube(c))


    OOO
    OOO
    OOO
YYY WWW GGG BBB
YYY WWW GGG BBB
YYY WWW GGG BBB
    RRR
    RRR
    RRR


In [123]:
import random

SOLVED_CUBE_STR = "OOOOOOOOOYYYWWWGGGBBBYYYWWWGGGBBBYYYWWWGGGBBBRRRRRRRRR"
#MOVES = ["L", "R", "U", "D", "F", "B", "M", "E", "S", "Si"]
MOVES = ["L", "R", "U", "D", "F", "B", "M", "S", "Si"]

def random_cube(num_moves: int = 1):
    """
    :return: A new scrambled Cube
    """
    scramble_moves = " ".join(random.choices(MOVES, k=num_moves))
    c = Cube(SOLVED_CUBE_STR)
    c.sequence(scramble_moves)
    return c

print(colored_cube(random_cube(1)))

    GGG
    OOO
    OOO
OYY WWW GGR BBB
OYY WWW GGR BBB
OYY WWW GGR BBB
    RRR
    RRR
    YYY


In [124]:
c = Cube("OOOOOOOOOYYYWWWGGGBBBYYYWWWGGGBBBYYYWWWGGGBBBRRRRRRRRR")
# print(c._color_list())

def cube_hash(cube: Cube):
  res = 0
  res_list=[]
  for piece in cube._color_list():
    # O: 1, Y: 2, W: 3, G: 4, B: 5, R: 6
    if(piece == "O"):
      res_list.append(1)
    elif(piece == "Y"):
      res_list.append(2)
    elif(piece == "W"):
      res_list.append(3)
    elif(piece == "G"):
      res_list.append(4)
    elif(piece == "B"):
      res_list.append(5)
    elif(piece == "R"):
      res_list.append(6)
  
  for i in range(6):
      res *= 9
      res += res_list[i]
  
  return res

c = random_cube(100)
print(colored_cube(c))
print("Cube Hash:", cube_hash(c))



    GBB
    YBW
    RGR
RRW YRY BOO GOW
GRW GYY WOO YGO
OWW YBG BYO YBW
    ORR
    RWB
    GGB
Cube Hash: 272856


In [125]:
cube = Cube("OOOOOOOOOYYYWWWGGGBBBYYYWWWGGGBBBYYYWWWGGGBBBRRRRRRRRR")
cube.S()
print(cube)
print(cube.is_solved())
cube.Si()
print(cube)
print(cube.is_solved())

    OOO
    YYY
    OOO
YRY WWW GOG BBB
YRY WWW GOG BBB
YRY WWW GOG BBB
    RRR
    GGG
    RRR
False
    OOO
    OOO
    OOO
YYY WWW GGG BBB
YYY WWW GGG BBB
YYY WWW GGG BBB
    RRR
    RRR
    RRR
True


In [126]:
import numpy as np

class Agent:
    def __init__(self, alpha=0.9, gamma=0.95, q_init=0.6):
        # alpha: The learning rate needs to be larger than 0 and smaller than 1
        # gamma: The reward discount. Needs to be larger than 0  and should be smaller than 1. Values close to 1 should work best.
        # q_init: The initial q values for each action and state.
        
        self.q = {}  # type: Dict[int, np.ndarray]
        self.action_history = []  # type: List[(int, int)]
        self.learning_rate = alpha
        self.value_discount = gamma
        self.q_init_val = q_init
        super().__init__()

    def get_q(self, hash_value: int) -> np.ndarray:
        # Returns the q values for the state with hash value of cube
        # hash_value: The hash value of the board state for which the q values should be returned
        # return: List of q values for the input state hash.

        #
        # Build Q table in a lazy manner, only adding a state when it is actually used for the first time
        #
        if hash_value in self.q:
            qvals = self.q[hash_value]
        else:
            qvals = np.full(9, self.q_init_val)
            self.q[hash_value] = qvals

        return qvals

    def get_action(self, cube: Cube) -> int:
        # Return the next action given the cube `board` cube on the current Q values
        # cube: The current cube state
        # return: The next action based on the current Q values for the input state
        
        hash_value = cube_hash(cube)  # type: int
        qvals = self.get_q(hash_value)  # type: np.ndarray
        m = np.argmax(qvals)  # type: int
        return m

    def action(self, cube: Cube):
        # Makes a action to solve the cube
        # cube: The cube to make a action on
        # return: Bool whether the cube is solved

        m = self.get_action(cube)
        self.action_history.append((cube_hash(cube), m))
        # cube.sequence(MOVES[m])
        cube.sequence(MOVES[m])

        solved = cube.is_solved()

        return solved, MOVES[m]

    def update_q(self, moves: int = 0, result: float = None):
        # Gets called after the game has finished. Will update the current Q function based on the actions.

        if(result == None):
            result =  len(self.action_history) / moves
    
        # if(len(self.action_history) == moves):
        #     result = 1
        # else:
        #     result = 0.5

        self.action_history.reverse()
        next_max = -1.0  # type: float

        for a in self.action_history:
            qvals = self.get_q(a[0])
            if next_max < 0:  # First time through the loop
                qvals[a[1]] = result
            else:
                qvals[a[1]] = qvals[a[1]] * (
                            1.0 - self.learning_rate) + self.learning_rate * self.value_discount * next_max

            next_max = max(qvals)

    def new_agent(self):
        self.action_history = []

In [127]:
def try_to_solve(cube: Cube, agent: Agent, num_moves: int, cancle_after: int) -> bool:
    agent.new_agent()
    
    for _ in range(cancle_after):
      solved, move = agent.action(cube)
      print(solved, move)
      if solved:
        agent.update_q(num_moves)
        return True, move

    print(agent.q)
    agent.update_q(num_moves, -1)
    return False, move

In [128]:
from datetime import datetime
import pandas as pd
# 
now = datetime.now().strftime("%d%m%Y_%H%M%S")
file_name = f'data/cube_{now}.csv'

def train(agent: Agent, num_moves: int = 1, cancle_after: int = 1, num_games: int = 10, silent: bool = False):

  row=[]
  rows=[]

  solved_count=0
  for _ in range(num_games):
    # cube = random_cube(num_moves)
    cube = Cube("OOOOOOOOOYYYWWWGGGBBBYYYWWWGGGBBBYYYWWWGGGBBBRRRRRRRRR")
    cube.S()
    row.append(cube_hash(cube))
    row.append("S")

    solved, move = try_to_solve(cube, agent, num_moves, cancle_after)

    row.append(move)
    row.append(agent.q)

    rows.append(row)
    row=[]
  
    if(solved):
      solved_count+=1

  
  df_cubes = pd.DataFrame(rows, columns=['Hash', 'Action', 'Move', "Q"])
  df_cubes.to_csv(file_name, index=False, sep="|")
  print(solved_count)

agent = Agent()
train(agent, 1, 1, 18)
a = pd.read_csv(file_name, sep="|")
a


False L
{66521: array([0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6])}
False R
{66521: array([-1. ,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6])}
False U
{66521: array([-1. , -1. ,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6])}
False D
{66521: array([-1. , -1. , -1. ,  0.6,  0.6,  0.6,  0.6,  0.6,  0.6])}
False F
{66521: array([-1. , -1. , -1. , -1. ,  0.6,  0.6,  0.6,  0.6,  0.6])}
False B
{66521: array([-1. , -1. , -1. , -1. , -1. ,  0.6,  0.6,  0.6,  0.6])}
False M
{66521: array([-1. , -1. , -1. , -1. , -1. , -1. ,  0.6,  0.6,  0.6])}
False S
{66521: array([-1. , -1. , -1. , -1. , -1. , -1. , -1. ,  0.6,  0.6])}
True Si
True Si
True Si
True Si
True Si
True Si
True Si
True Si
True Si
True Si
10


,Hash,Action,Move,Q
0,66521,S,L,"{66521: array([-1., -1., -1., -1., -1., -1., -1., -1., 1.])}"
1,66521,S,R,"{66521: array([-1., -1., -1., -1., -1., -1., -1., -1., 1.])}"
2,66521,S,U,"{66521: array([-1., -1., -1., -1., -1., -1., -1., -1., 1.])}"
3,66521,S,D,"{66521: array([-1., -1., -1., -1., -1., -1., -1., -1., 1.])}"
4,66521,S,F,"{66521: array([-1., -1., -1., -1., -1., -1., -1., -1., 1.])}"
5,66521,S,B,"{66521: array([-1., -1., -1., -1., -1., -1., -1., -1., 1.])}"
6,66521,S,M,"{66521: array([-1., -1., -1., -1., -1., -1., -1., -1., 1.])}"
7,66521,S,S,"{66521: array([-1., -1., -1., -1., -1., -1., -1., -1., 1.])}"
8,66521,S,Si,"{66521: array([-1., -1., -1., -1., -1., -1., -1., -1., 1.])}"
9,66521,S,Si,"{66521: array([-1., -1., -1., -1., -1., -1., -1., -1., 1.])}"


In [129]:
from datetime import datetime
# 
now = datetime.now().strftime("%d%m%Y_%H%M%S")
print("date and time =", now)

date and time = 14022022_111138


In [130]:
import pandas as pd

df = pd.read_csv('data/test.csv')
df

,a,b,c
0,1,2,3
1,4,5,6


In [131]:
cities = pd.DataFrame([['Sacramento', 'California'], ['Miami', 'Florida']], columns=['City', 'State'])

cubes = [random_cube(4) for _ in range(5) ]
# [print(cube) for cube in cubes]
cube_hashes = [cube_hash(cube) for cube in cubes]
colored_cubes = [colored_cube(cube) for cube in cubes]
df = [[colored_cubes[i], cube_hashes[i]] for i in range(len(cubes))]
# print(df)

df_cubes = pd.DataFrame(df, columns=['Cube', 'Hash'])

df_cubes.to_csv('data/cube.csv', index=False, sep="|")
a = pd.read_csv('data/cube.csv', sep="|")

print(a)
from IPython.display import display, HTML
# display(HTML(a.to_clipboard()))
# print(a.to_json())
# print(a)
# for column in a:
#     print(a[column])
# for _ in range(len(cubes)):
#   for name, values in a.iteritems():
#       print('{name}: {value}'.format(name=name, value=values[0]))
# for (columnName, columnData) in a.iteritems():
#     print('Column Name : ', columnName)
#     print('Column Contents : ', columnData.values[0])
# for x in a["Cube"]:
#     print(x)
!pip3 install ansi2html
pd.set_option('display.max_colwidth', None)
from IPython.display import display, HTML
from ansi2html import Ansi2HTMLConverter
conv = Ansi2HTMLConverter()

def pretty_print(df):
    for i, x in enumerate(df["Cube"]):
        df["Cube"][i] = conv.convert("".join(x))
        # print(display(HTML(conv.convert(x))))

    html_df = df.to_html()
    newline_df = html_df.replace("\\n","<br>")
    # colored_df = conv.convert(newline_df)


    return  display(HTML(newline_df))

display(HTML('<h1>Hello, world!</h1>'))
pretty_print(a)
# display(HTML(a.to_html()))

# 


# conv = Ansi2HTMLConverter()
# ansi = "".join(colored_cube(random_cube(1)))
# html = conv.convert(ansi)
# display( HTML( html ) )

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Cube    Hash
0      ROO\n    YRY\n    ROO\nYOY BWW GRG BBW\nRGW YWW GYG BBG\nYGY BWW GYG BBW\n    ORR\n    OOB\n    ORR  361802
1      ROO\n  

ModuleNotFoundError: No module named 'ansi2html'